In [9]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
pd.set_option('float_format', '{:f}'.format)

In [2]:
cd ..

/Users/alphonsowoodbury/DS/mod2project/flatiron_mod2_project_kch


In [3]:
#importing data
df = pd.read_csv('data/kc_house_data.csv')
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


## Cleaning

In [6]:
#changing question marks to 0.0
df = df.replace('?', 0.0)
df.view = df.view.replace(np.nan, 0)
#changing all column object types to floats (except date column)
df.loc[:, df.columns != 'date'] = df.loc[:,df.columns != 'date'].astype('float')

#changing all 0.0 in sqft_basement column, yr_renovated, and waterfront columns to NaN values
df['sqft_basement'] = df['sqft_basement'].replace(0.0 , np.nan)
df['waterfront'] = df.waterfront.replace(0.0, np.nan)
df['yr_renovated'] =df['yr_renovated'].replace(0.0, np.nan)
#changing date column to datetime values
df['date'] = pd.to_datetime(df['date'])
df['yr_sold'] = df['date'].dt.to_period('Y')
df = df.drop(['id'],axis=1)

## Feature Engineering

In [ ]:
#creating has_basement, has_waterfront, has_renovation columns with True/False values
df['has_waterfront'] = df['waterfront'].notnull().astype('int64')
df['has_basement'] = df['sqft_basement'].notnull().astype('int64')
df['has_renovation'] = df['yr_renovated'].notnull().astype('int64')
#creating eff_built column (which updates built year depending on whether it was renovated or not)
df.loc[df['yr_renovated'].notnull(), 'eff_built'] = 2020 - df['yr_renovated']
df.loc[df['yr_renovated'].isnull(), 'eff_built'] = 2020 -df['yr_built']
df.eff_built = df.eff_built.astype('int64')

#correcting data types
##discrete vars as int
df.bedrooms = df.bedrooms.astype('int64')
df.bathrooms = df.bathrooms.astype('int64')
df.floors = df.floors.astype('int64')
df.zipcode = df.zipcode.astype('int64')
df.condition = df.condition.astype('int64')
df.grade = df.grade.astype('int64')
df.view = df.view.astype('int64')

#categoricals as obj
df.zipcode = df.zipcode.astype('object')

#drop pre-processed columns
df = df.drop(['date','waterfront','sqft_above','sqft_basement','yr_built','yr_renovated'],axis=1)


In [11]:
df.describe(include='all')

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold
count,21597,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,146.000000,21597.000000,21597.000000,...,21597.000000,8317.000000,21597.000000,744.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597
unique,372,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2
top,2014-06-23 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2014
freq,142,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,14622
first,2014-05-02 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
last,2015-05-27 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
mean,NaN,540296.573506,3.373200,2.115826,2080.321850,15099.408760,1.494096,1.000000,0.233181,3.409825,...,1788.596842,741.928700,1970.999676,1995.928763,98077.951845,47.560093,-122.213982,1986.620318,12758.283512,NaN
std,NaN,367368.140101,0.926299,0.768984,918.106125,41412.636876,0.539683,0.000000,0.764673,0.650546,...,827.759761,404.765113,29.375234,15.599946,53.513072,0.138552,0.140724,685.230472,27274.441950,NaN
min,NaN,78000.000000,1.000000,0.500000,370.000000,520.000000,1.000000,1.000000,0.000000,1.000000,...,370.000000,10.000000,1900.000000,1934.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,NaN
25%,NaN,322000.000000,3.000000,1.750000,1430.000000,5040.000000,1.000000,1.000000,0.000000,3.000000,...,1190.000000,450.000000,1951.000000,1987.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000,NaN


## Export

In [ ]:
cd Data

In [ ]:
df.to_csv('kc_cleaned.csv',index=False)
df

In [ ]:
df.info()